In [1]:
from pyhmy import cli
import json
import pandas as pd
import numpy as np
import os
import time

In [2]:
env = cli.download()
cli.environment.update(env)

Saved harmony binary to: `/Users/yishuang/Desktop/log_analysis/notebooks/staking_dashboard/bin/hmy`


In [3]:
path = os.getcwd() + "/bin/hmy"
cli.set_binary(path)

True

In [4]:
def R1_test():
    committees = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io utility committees"))
    harmony_nodes = []
    for k,v in committees['result']['current']['quorum-deciders'].items():
        for i in v['committee-members']:
            if i['is-harmony-slot'] == True:
                harmony_nodes.append(i['earning-account'])
    
    # no harmony nodes should have validator information
    num = 0
    for i in harmony_nodes:
        try:
            cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i)
        except:
            num += 1
    assert num == len(harmony_nodes)
    return True
            

In [5]:
R1_test()

True

In [6]:
def E1_test():
    committees = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io utility committees"))
    slot = committees['result']['current']['external-slot-count']
    validator = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator all"))['result'] 
    epoch = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']['epoch']
    # get top #slots nodes who are eligible to elected next epoch
    new = []
    stake = dict()
    for i in validator:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        if info['epos-status'] == 'currently elected and signing enough blocks to be eligible for election next epoch' or info['epos-status'] == 'eligible to be elected next epoch':
            new.append(i)
            stake[i] = info['total-delegation']
    if len(new) > slot:
        sorted_stake = sorted(stake.items(), key=lambda kv: kv[1], reverse = True)
        new = [kv[0] for kv in sorted_stake[:slot]]
    
    # wait for epoch changes
    new_epoch = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']['epoch']
    while new_epoch == epoch:
        time.sleep(8)
        new_epoch = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain latest-header"))['result']['epoch']
    
    # check whether the eligible validators are selected
    for i in new:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        if info['epos-status'] == 'currently elected and signing enough blocks to be eligible for election next epoch' or \
        info['epos-status'] == 'currently elected and not signing enough blocks to be eligible for election next epoch':
            print("validator " + i + "who is eligible to be selected is not selected")

            
        
        

In [7]:
E1_test()

validator one1m6j80t6rhc3ypaumtsfmqwjwp0mrqk9ff50prhwho is eligible to be selected is not selected
validator one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9who is eligible to be selected is not selected
validator one103q7qe5t2505lypvltkqtddaef5tzfxwsse4z7who is eligible to be selected is not selected
validator one10fjqteq6q75nm62cx8vejqsk7mc8t5hle8ewnlwho is eligible to be selected is not selected
validator one1vzsj3julf0ljcj3hhxuqpu6zvadu488zfrtttzwho is eligible to be selected is not selected
validator one1marnnvc8hywmfxhrc8mtpjkvvdt32x9kxtwkvvwho is eligible to be selected is not selected
validator one1y5n7p8a845v96xyx2gh75wn5eyhtw5002lah27who is eligible to be selected is not selected
validator one1egemh5e9xjy3x8d3cq0kq7mw4sw4jjwgkc7axswho is eligible to be selected is not selected
validator one1km7xg8e3xjys7azp9f4xp8hkw79vm2h3f2ladewho is eligible to be selected is not selected
validator one17tj2jjehdlg8xfgp48xpeyqur2qf6nvs88jvyuwho is eligible to be selected is not selected
validator 

the median stake calcualtion should be applied to all the contested validators, who are eligible for elected next epoch. 
- before election, get the top 200 eligible nodes, and calculate the median;
- after election, calculate the median for the elected node. both the two status:
    - currently elected and signing enough blocks to be eligible for election next epoch 
    - currently elected and not signing enough blocks to be eligible for election next epoch
    
Q: if they are elected, they have the metrics, not elected no metrics?

In [8]:
def M5_test():
    median = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain median-stake"))
    validator = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator all"))['result'] 
    
    # get the median stake and the upper and lower level 
    median_stake = float(median['result']['epos-median-stake'])
    lower = median_stake - 0.15*median_stake
    upper = median_stake + 0.15*median_stake
    print("median stake is " + median['result']['epos-median-stake'])
    print("lower bond is " + str(lower))
    print("upper bond is " + str(upper))
    print("===========================")
    print("")
    # check every validator's effective stake
    for i in validator:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        if info['metrics']:
            for s in info['metrics']['by-shard']:
                if float(s['effective-stake']) > upper or float(s['effective-stake']) < lower:
                    print("validator: " + i + " bls public key: " + str(s['bls-public-key']) +\
                          " shard: " + str(s['shard-id'])) 
                    print("effective stake is out of range. The effective stake is " + s['effective-stake'])
                    print("")

In [9]:
M5_test()

median stake is 200000000000000000000000.000000000000000000
lower bond is 1.7e+23
upper bond is 2.2999999999999997e+23

validator: one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 bls public key: 7ab21e88f5d17712e30dfc9a7acdb7e0e784fabf2bf76d5f83d8a0269dd47f38d8f454cb7ece5921027a9b68d806578f shard: 3
effective stake is out of range. The effective stake is 230000000000000000000000.000000000000000000

validator: one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 bls public key: bd8a0ab99bc6468b10992846f74b9f4c23ed1ce9a89848a4edba15bbef10906091880e9f10b3b0e53000c2b4f934cb97 shard: 3
effective stake is out of range. The effective stake is 230000000000000000000000.000000000000000000

validator: one103q7qe5t2505lypvltkqtddaef5tzfxwsse4z7 bls public key: 86dc2fdc2ceec18f6923b99fd86a68405c132e1005cf1df72dca75db0adfaeb53d201d66af37916d61f079f34f21fb96 shard: 2
effective stake is out of range. The effective stake is 230000000000000000000000.000000000000000000

validator: one12d5a58rcpyf8chlcd2my8r8ns572uppetm

question: in the same epoch, will the delegation change

In [10]:
def get_median(lst):
    n = len(lst) 
    lst.sort() 
    if n % 2 == 0: 
        median1 = lst[n//2] 
        median2 = lst[n//2 - 1] 
        median = (median1 + median2)/2
    else: 
        median = lst[n//2] 
    return median

In [13]:
def M2_test():
    median = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain median-stake"))
    validator = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator all"))['result'] 
    median_stake = float(median['result']['epos-median-stake'])
    
    elected = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator elected"))
    stake = []
    for i in validator:
        info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
        stake.append(info['total-delegation'])
    cal_median = get_median(stake)
#     assert median == cal_median
    print("manually calculated median stake: " + str(cal_median))
    print("harmony apr call median stake: " + str(median_stake))

In [14]:
M2_test()

manually calculated median stake: 4e+23
harmony apr call median stake: 2e+23
